## IMPORT STATEMENTS

In [5]:
# Import Statements
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

## SET UP FOR DEVICE AGNOSTIC CODE

In [6]:
# Check is GPU is available
torch.cuda.is_available()

False

In [7]:
# Setup device agnostic code
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cpu'

## DATA LOADING AND PREPROCESSING

In [8]:
# Mount the google drive (RUN ONLY IN COLAB)
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [9]:
# Set up data directory path
TRAIN_DATA_DIR = "/content/drive/My Drive/Data/inaturalist_12K/train"
TEST_DATA_DIR = "/content/drive/My Drive/Data/inaturalist_12K/val"

In [17]:
image_size = (300, 300)
data_augment = False
valset_size = 0.2
batch_size = 8


# Define transformations to be applied
base_transforms = [
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
]


# If Augmentation is needed, add them to transform list
train_transforms = base_transforms.copy()
if data_augment:
    train_transforms += [
        transforms.RandomHorizontalFlip(0.2),
        transforms.RandomRotation(degrees=20),
        transforms.RandomApply([transforms.ColorJitter(0.2, 0.2, 0.2, 0.2)], p=0.1),
        transforms.RandomApply([transforms.GaussianBlur(3)], p=0.1)
    ]

# Apply Compose to the transform lists
train_transform = transforms.Compose(train_transforms)
test_transform = transforms.Compose(base_transforms)

# Dowload the total_train data.
total_trainset = torchvision.datasets.ImageFolder(root = TRAIN_DATA_DIR, transform=train_transform)

# Split the total_train data into train data and val data
targets = [target for _, target in total_trainset.samples]

train_indices, val_indices = train_test_split(
                                range(len(total_trainset)),
                                test_size=valset_size,
                                stratify=targets,
                                random_state=42
                                )

trainset = torch.utils.data.Subset(total_trainset, train_indices)
valset = torch.utils.data.Subset(total_trainset, val_indices)


#Download the test data
testset = torchvision.datasets.ImageFolder(root=TEST_DATA_DIR, transform=test_transform)



# Create the dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                        shuffle=False, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=2)

## MODEL CLASS

In [18]:
class CNNModel(nn.Module):
    def __init__(self, image_size, in_channels=3, num_classes=10,
                 num_filters=[16, 32, 64, 128, 256], kernel_size=3,
                 activation_fn=nn.ReLU, fc_layer_size=2048,
                 batchnorm=False, dropout=0.0):

        super().__init__()

        h, w = image_size

        # Block 1
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=num_filters[0], kernel_size=kernel_size)
        h, w = h - kernel_size + 1, w - kernel_size + 1
        self.batchnorm1 = nn.BatchNorm2d(num_filters[0]) if batchnorm else nn.Identity()
        self.activation1 = activation_fn()
        self.dropout1 = nn.Dropout2d(dropout) if dropout!=0 else nn.Identity()
        self.maxpool1 = nn.MaxPool2d(2, 2)
        h, w = h//2, w//2

        # Block 2
        self.conv2 = nn.Conv2d(in_channels=num_filters[0], out_channels=num_filters[1], kernel_size=kernel_size)
        h, w = h - kernel_size + 1, w - kernel_size + 1
        self.batchnorm2 = nn.BatchNorm2d(num_filters[1]) if batchnorm else nn.Identity()
        self.activation2 = activation_fn()
        self.dropout2 = nn.Dropout2d(dropout) if dropout!=0 else nn.Identity()
        self.maxpool2 = nn.MaxPool2d(2, 2)
        h, w = h//2, w//2

        # Block 3
        self.conv3 = nn.Conv2d(in_channels=num_filters[1], out_channels=num_filters[2], kernel_size=kernel_size)
        h, w = h - kernel_size + 1, w - kernel_size + 1
        self.batchnorm3 = nn.BatchNorm2d(num_filters[2]) if batchnorm else nn.Identity()
        self.activation3 = activation_fn()
        self.dropout3 = nn.Dropout2d(dropout) if dropout!=0 else nn.Identity()
        self.maxpool3 = nn.MaxPool2d(2, 2)
        h, w = h//2, w//2

        # Block 4
        self.conv4 = nn.Conv2d(in_channels=num_filters[2], out_channels=num_filters[3], kernel_size=kernel_size)
        h, w = h - kernel_size + 1, w - kernel_size + 1
        self.batchnorm4 = nn.BatchNorm2d(num_filters[3]) if batchnorm else nn.Identity()
        self.activation4 = activation_fn()
        self.dropout4 = nn.Dropout2d(dropout) if dropout!=0 else nn.Identity()
        self.maxpool4 = nn.MaxPool2d(2, 2)
        h, w = h//2, w//2

        # Block 5
        self.conv5 = nn.Conv2d(in_channels=num_filters[3], out_channels=num_filters[4], kernel_size=kernel_size)
        h, w = h - kernel_size + 1, w - kernel_size + 1
        self.batchnorm5 = nn.BatchNorm2d(num_filters[4]) if batchnorm else nn.Identity()
        self.activation5 = activation_fn()
        self.dropout5 = nn.Dropout2d(dropout) if dropout!=0 else nn.Identity()
        self.maxpool5 = nn.MaxPool2d(2, 2)
        h, w = h//2, w//2

        # Flattening layer
        self.flatten = nn.Flatten()

        # Fully connected layers
        self.fc_layer = nn.Linear(in_features=num_filters[4] * h * w, out_features=fc_layer_size)
        self.drop_fc = nn.Dropout(dropout) if dropout!=0 else nn.Identity()
        self.act_fc = activation_fn()

        # Output layer
        self.out = nn.Linear(in_features=fc_layer_size, out_features=num_classes)

    def forward(self, x):
        # Block 1
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.activation1(x)
        x = self.dropout1(x)
        x = self.maxpool1(x)

        # Block 2
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = self.activation2(x)
        x = self.dropout2(x)
        x = self.maxpool2(x)

        # Block 3
        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = self.activation3(x)
        x = self.dropout3(x)
        x = self.maxpool3(x)

        # Block 4
        x = self.conv4(x)
        x = self.batchnorm4(x)
        x = self.activation4(x)
        x = self.dropout4(x)
        x = self.maxpool4(x)

        # Block 5
        x = self.conv5(x)
        x = self.batchnorm5(x)
        x = self.activation5(x)
        x = self.dropout5(x)
        x = self.maxpool5(x)

        # Flatten
        x = self.flatten(x)

        # Fully connected layers
        x = self.fc_layer(x)
        x = self.drop_fc(x)
        x = self.act_fc(x)

        # Output layer
        x = self.out(x)

        return x

In [19]:
model = CNNModel(image_size)
model.to(device)

CNNModel(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm1): Identity()
  (activation1): ReLU()
  (dropout1): Identity()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm2): Identity()
  (activation2): ReLU()
  (dropout2): Identity()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm3): Identity()
  (activation3): ReLU()
  (dropout3): Identity()
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm4): Identity()
  (activation4): ReLU()
  (dropout4): Identity()
  (maxpool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm5): Identit

In [ ]:
def train_model(model, trainloader, valloader, optimizer, criterion, device, epochs=10):
  pass

In [20]:
def evaluate_model(model, testloader, criterion, device):
  pass